Here we want to run consensus on all the methods
inputs: list of modules in all methods


In [2]:
import os
import pandas as pd
from pathlib import Path, PosixPath
import matplotlib.pyplot as plt

In [3]:
workdir = os.path.dirname(os.getcwd())
input_dir = Path(f"{workdir}/3_module_expansion/data/categories_methods")
ppi_dir = Path("/home/bbc8731/diseasemodulediscovery/tests/uniprot_ppi.csv")
output_dir = Path("/home/bbc8731/HSV/4_consensus_module_construction/data/consensus_results")
output_dir.mkdir(parents=True, exist_ok=True)

In [4]:
for d in input_dir.glob('*'):
    module_dir = d / 'modules' / 'tsv_nodes'
    
    # there is a log folder that we need to pass
    if not module_dir.is_dir():
        continue

    # print(module_dir)
    # module_files = [p.name for p in module_dir.glob("*.tsv") if ("firstneighbor" not in p.name.lower() and "domino" not in p.name.lower())]    
    module_files = [p.name for p in module_dir.glob('*.tsv') if "firstneighbor" not in p.name]    
    tissues = {f.split(".", 2)[1] for f in module_files}

    for t in tissues:
        gene_list = []
        for f in module_files:
            if f".{t}." in f:
                module_t = pd.read_csv(module_dir / f, sep="\t")
                gene_list.extend(module_t["name"].dropna().unique())
        gene_counts = (pd.Series(gene_list).value_counts().reset_index())
        gene_counts.columns = ["gene", "n_methods"]
        out_dir = d/"consensus"
        out_dir.mkdir(exist_ok=True)
        out_file = out_dir / f"{t}.tsv"
        gene_counts.to_csv(out_file, sep="\t", index=False)